# build

> An even newer description.

In [ ]:
#| default_exp core

In [1]:
#| hide
from nbdev.showdoc import *

In [2]:
#| export
import os
import toml
import argparse
#from .core import read_base_config


def replace_inserts_with_toml_data(input_file, toml_file, output_file):
    # Replace \INSERT{var} with corresponding TOML data
    # Load TOML data
    with open(toml_file, 'r') as f:
        data = toml.load(f)

    # Read input text file
    with open(input_file, 'r') as f:
        content = f.read()

    start_pos = 0
    while True:
        start_idx = content.find(r'\INSERT{', start_pos)
        if start_idx == -1:
            break
        end_idx = content.find('}', start_idx)
        if end_idx == -1:
            break

        var_name = content[start_idx + 8:end_idx]
        replacement_value = data.get(var_name, '')  # Use a default value if var_name is not in TOML data
        print(f"Replacing {var_name} with {replacement_value}")
        content = content[:start_idx] + replacement_value + content[end_idx + 1:]
        start_pos = start_idx + len(replacement_value)

    # Write modified content to output .tex file
    with open(output_file, 'w') as f:
        f.write(content)

    return content

def run_lowdown(input_file, output_file):
    lowdown_cmd = f'''lowdown -stlatex --parse-no-intraemph --parse-no-super {input_file} | sed -n '/\\\\begin{{document}}/,/\\\\end{{document}}/p' | sed '1d;$d' > {output_file}'''
    print(lowdown_cmd)
    os.system(lowdown_cmd)



def lowdown_postprocess(input_file, output_file):

    # Read input text file
    with open(input_file, 'r') as f:
        content = f.read()

    #if 'textbackslash{}' in content:
    #    print("textbackslash{} found in input file.")
    #else:
    #    print("Warning: textbackslash{} not found in input file. This may cause problems.")

    lowdown_postprocessing = { 
        '\emph{': '*', # order matters
        'textbackslash{}': '',
        '\{': '{',
        '\}': '}',
        '\#': '#',
        '\$': '$',
        '\%': '%',
        '\&': '&',
        '\_': '_',
        '\\textasciicircum{}': '^',
    }

    for key, value in lowdown_postprocessing.items():
        if key in content:
            print(f"Replacing {key} with {value}")
            content = content.replace(key, value)

    # Write modified content to output .tex file
    with open(output_file, 'w') as f:
        f.write(content)



def main():

    base_config = read_base_config()

    parser = argparse.ArgumentParser(description="Replace \INSERT{var} in a text file using data from a TOML file.")
    parser.add_argument(
        "--input_file", 
        help="Path to the input text file containing \INSERT{var} placeholders.",
        default=base_config['repro']['files']['input']
    )
    parser.add_argument(
        "--toml_file", 
        help="Path to the TOML file containing replacement data.",
        default=base_config['repro']['files']['dynamic']
    )
    parser.add_argument(
        "--output_file", 
        help="Path to the output .tex file.",
        default=base_config['repro']['files']['output']
    )
    
    args = parser.parse_args()
    
    content = replace_inserts_with_toml_data(args.input_file, args.toml_file, 'reproduce/tmp/lowdown_input.md')

    print('Running lowdown')    
    run_lowdown('reproduce/tmp/lowdown_input.md', 'reproduce/tmp/lowdown_output.tex')

    print('Postprocessing lowdown output')
    lowdown_postprocess('reproduce/tmp/lowdown_output.tex', 'reproduce/tmp/lowdown_processed.tex')

    print('Inserting lowdown output into template')
    with open('reproduce/tmp/lowdown_processed.tex', 'r') as f:
        content = f.read()

    with open(base_config['repro']['files']['latex_template'], 'r') as f:
        template = f.read()

    template = template.replace('\\INSERT{lowdown_content}', content)

    with open(args.output_file, 'w') as f:
        f.write(template)

    print(f"Output written to {args.output_file}")


if __name__ == "__main__":
    main()


# Example Usage
#replace_inserts_with_toml_data('input.txt', 'data.toml', 'output.tex')


FileNotFoundError: [Errno 2] No such file or directory: 'reproduce/config.toml'

In [3]:
#| hide
import nbdev; nbdev.nbdev_export()

InterpolationMissingOptionError: Bad value substitution: option 'lib_name' in section 'DEFAULT' contains an interpolation key 'repo' which is not a valid option name. Raw value: '%(repo)s'